Перед тем как построить свою модель, дата-сайентист проходит через огромный этап под названием **Предобработка данных и их подготовка к анализу и подаче в модель**.

Под **предобработкой** понимаются следующие этапы работы с данными:

* очистка данных от аномальных значений (выбросов);
* работа с пропущенными значениями;
* удаление признаков, которые не несут полезной информации;
* создание новых признаков;
* преобразование признаков и приведение данных к необходимому для анализа и модели формату.

Почему этот этап так важен? Eсли данные плохо подготовлены, то и результат прогнозирования даже самой мощной в мире нейронной сети будет сильно разниться с действительностью.

## Feature Engineering

Одним из этапов подготовки данных является удаление, преобразование и создание столбцов таблицы.

Такой подход часто называют **Feature Engineering**, или генерацией признаков (фичей).

На самом деле, в этом термине заложен более глубокий смысл, ведь *Feature Engineering* — это целая методология получения более качественных и более производительных моделей за счёт манипуляций над данными. Специалисты часто называют данную методологию настоящим искусством, которое может быть освоено лишь с годами практики решения задач, ведь необходимо быть экспертом в исследуемой предметной области, чтобы понимать, как признаки влияют друг на друга и какое преобразование стоит к ним применить. 

In [94]:
# Импортируем Pandas
import pandas as pd

# прочитаем наш csv-файл в DataFrame
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
# выведем первые пять строк таблицы
melb_data.head()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941"


## Создание копии первоначальной таблицы

Чтобы не переопределять переменную melb_data и тем самым не повредить первоначальный DataFrame:

In [95]:
# Cоздадим копию melb_df с помощью метода copy():
melb_df = melb_data.copy()
melb_df.head()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941"


## Удаление столбцов

Среди списка базовых операций над столбцами в Pandas важное место занимает возможность удаления столбцов из таблицы. 

Представим, что мы хотим построить модель, которая бы предсказывала цену объекта недвижимости в Мельбурне. Даже не будучи профессиональными риелторами, мы можем легко сделать следующие выводы:

* цена объекта никак не зависит от его порядкового номера (столбец index);
* признак, описывающий долготу и широту в виде кортежа Coordinates, дублирует информацию, представленную в столбцах Longitude и Lattitude.

За удаление строк и столбцов в таблице отвечает метод drop(). Основные параметры: 

*  labels — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;
*  axis — ось совершения операции, axis=0 — удаляются строки, axis=1 — удаляются столбцы;
*  inplace — если параметр выставлен на True, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает; если на False — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; по умолчанию параметр равен False.

In [96]:
# Удалим столбцы index и Coordinates из таблицы с помощью метода drop(). 
# Выведем первые пять строк таблицы и убедимся, что всё прошло успешно.
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
melb_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,...,1,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067,...,1,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067,...,2,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067,...,2,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067,...,1,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019


In [ ]:
# Альтернативный вариант:

melb_df.drop(['index','Coordinates'],axis=1,inplace=True)
melb_df.head()


## Математические операции со столбцами

In [97]:
# Cоздадим переменную total_rooms, в которой будем хранить общее количество комнат в здании. 
# Для этого выполним сложение столбцов с количеством комнат, ванн и спален:

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
display(total_rooms)

0         5
1         5
2         8
3         8
4         8
         ..
13575    10
13576     8
13577     8
13578     9
13579     9
Length: 13580, dtype: int64

In [98]:
# Bведём признак MeanRoomsSquare, 
# который соответствует средней площади одной комнаты для каждого объекта. 
# Для этого разделим площадь здания на полученное ранее общее количество комнат:

melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'])

0        25.200000
1        15.800000
2        18.750000
3        15.750000
4        17.750000
           ...    
13575    12.600000
13576    16.625000
13577    15.750000
13578    17.444444
13579    12.444444
Name: MeanRoomsSquare, Length: 13580, dtype: float64

In [99]:
# Bведём признак — AreaRatio, 
# коэффициент соотношения площади здания (BuildingArea) и площади участка (Landsize). 
# Для этого разницу двух площадей поделим на их сумму:

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])

0       -0.231707
1       -0.327660
2        0.056338
3        0.145455
4        0.083969
           ...   
13575   -0.676093
13576   -0.429185
13577   -0.551601
13578   -0.693060
13579   -0.527426
Name: AreaRatio, Length: 13580, dtype: float64

Что показывает такой коэффициент? Если присмотреться, можно увидеть, что AreaRatio лежит в интервале от -1 до 1.

Рассмотрим три случая, чтобы понять его значение:

*  Если рассматриваемые площади равны, то числитель дроби зануляется и коэффициент тоже равен 0.
*  Если одна из площадей начинает доминировать над другой, то коэффициент начинает расти в отрицательную сторону, если площадь участка больше площади здания, и в положительную сторону, если наоборот.
*  Наконец, в предельном случае, если площадь здания равна 0, то числитель дроби равен знаменателю со знаком минус, а коэффициент равен -1, а если площадь участка равна 0, то числитель дроби равен знаменателю со знаком плюс, а коэффициент равен 1.

Таким образом, значение в столбце AreaRatio служит своеобразным указателем соотношения площадей объекта недвижимости. Для пустырей — участков без строений — он будет равен -1, для домов без территории — 1, во всех остальных случаях мы можем увидеть, какая площадь больше — здания или участка.

In [100]:
# Вычислим квадрат цены объекта недвижимости 
price_square = melb_df['Price'] **2

Задан DataFrame customer_df, содержащий столбцы:
* cust_id — идентификатор клиента;
* cust_age — возраст клиента (точкой отсчёта возраста считается 2021 год);
* cust_sale — персональная скидка клиента;
* cust_year_birth — год рождения клиента;
* cust_order — сумма заказа клиента.

In [101]:
def delete_columns(df, col=[]):
    for cc in col:
        if cc not in df.columns:
            return None
    return df.drop(col, axis=1)
customer_df = pd.DataFrame({
        'number': [0, 1, 2, 3, 4],
        'cust_id': [128, 1201, 9832, 4392, 7472],
        'cust_age': [13, 21, 19, 21, 60],
        'cust_sale': [0, 0, 0.2, 0.15, 0.3],
        'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
        'cust_order': [1400, 14142, 900, 1240, 8430]
    })



Какие столбцы не несут полезной информации/дублируют информацию из других столбцов и поэтому могут быть удалены?

Напишите функцию delete_columns(df, col=[]), которая удаляет столбцы из DataFrame и возвращает новую таблицу. Если одного из указанных столбцов не существует в таблице, то функция должна возвращать None. Удалите выбранные вами столбцы из таблицы customer_df.

Для удаления столбцов используется метод drop() с параметром axis=1. В него передаётся список столбцов, подлежащих удалению.

In [ ]:
def delete_columns(df, col=[]):
    for cc in col:
        if cc not in df.columns:
            return None
    return df.drop(col, axis=1)
customer_df = pd.DataFrame({
        'number': [0, 1, 2, 3, 4],
        'cust_id': [128, 1201, 9832, 4392, 7472],
        'cust_age': [13, 21, 19, 21, 60],
        'cust_sale': [0, 0, 0.2, 0.15, 0.3],
        'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
        'cust_order': [1400, 14142, 900, 1240, 8430]
    })

print(delete_columns(col = ['cust_sale'], df = customer_df))

   number  cust_id  cust_age  cust_year_birth  cust_order
0       0      128        13             2008        1400
1       1     1201        21             2000       14142
2       2     9832        19             2002         900
3       3     4392        21             2000        1240
4       4     7472        60             1961        8430


In [ ]:
import pandas as pd
import numpy as np

def delete_columns(df, col=[]):
    for cc in col:
        if cc not in df.columns:
            return None
    return df.drop(col, axis=1)

 
test_df_2 = pd.DataFrame({
            'number2': [0, 1, 2, 3, 4],
            'cust_id2': [128, 1201, 9832, 4392, 7472],
            'cust_age2': [13, 21, 19, 21, 60],
            'cust_sale2': [0, 0, 0.2, 0.15, 0.3],
            'cust_year_birth2': [2008, 2000, 2002, 2000, 1961],
            'cust_order2': [1400, 14142, 900, 1240, 8430]
        })


print(delete_columns(col = ['cust_sale2', 'cust_sale2'], df = test_df_2))


   number2  cust_id2  cust_age2  cust_year_birth2  cust_order2
0        0       128         13              2008         1400
1        1      1201         21              2000        14142
2        2      9832         19              2002          900
3        3      4392         21              2000         1240
4        4      7472         60              1961         8430


Задан DataFrame countries_df, содержащий следующие столбцы: название страны, население (population) в миллионах человек и площадь страны (square) в квадратных километрах.

In [ ]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})

countries_df['population density'] = (countries_df['population'] * 1000000) / countries_df['square']
print(round(countries_df['population density'].mean(), 2))

84.93


## Признаки даты и времени 

→ При генерации новых признаков очень ценным может стать временной признак (признак даты и времени). Это особый тип данных, с которым приходится сталкиваться в большинстве задач по обработке данных. В реальных задачах часто нужно сравнивать даты, выделять день недели или час, вычислять различные интервалы между датами. 

Например, в наших данных об объектах недвижимости есть признак даты продажи — столбец Date. Из данного признака можно выделить массу полезной информации, например год, месяц или день недели продажи имущества. На рынке недвижимости, как известно, присутствует сезонность: есть периоды, когда недвижимость покупается чаще, а есть интервалы времени, когда рынок претерпевает застой, поэтому было бы неплохо учитывать эту сезонность при анализе рынка.

## Format Datetime

Формат времени из разных систем может отличаться:

* 2018-11-09 15:45:21;
* 11/09/2018 3:45:20 PM;
* 2018-11-09T15:45:21.2984.

Для всех этих случаев необходимо задавать формат распознавания дат и уметь сравнивать их между собой. Для этого был создан единый способ обозначения даты и времени. 

Таким форматом в Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

In [102]:
# В данном DataFrame дата записана в виде DD/MM/YYYY
display(melb_df['Date'])


0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат datetime, можно воспользоваться функцией pandas.to_datetime(). В нашем случае в функции нужно указать параметр dayfirst=True, который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime, передав его в эту функцию:

In [109]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

В результате мы переопределяем признак Date в формат datetime. При этом так как в изначальном варианте время не было указано, то и после преобразования оно опускается.

Стоит обратить внимание, что изменился тип данных для столбца Date, теперь его тип — datetime64.

## Выделение атрибутов datetime

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

* date — дата;
* year, month, day — год, месяц, день;
* time — время;
* hour, minute, second — час, минута, секунда;
* dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
* day_name — название дня недели;
* dayofyear — порядковый день года;
* quarter — квартал (интервал в три месяца).

Аксессор — это атрибут столбца, хранящий переменные, которые были строковым представлением времени, а затем были изменены с помощью pd.to_datetime().

Например, обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

In [110]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int64
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


Так как модальных значений в столбце может быть несколько, метод mode() возвращает объект Series, даже если мода в данных только одна. Чтобы сохранить стилистику вывода информации о годе продажи и выводить только число, а не Series, мы обращаемся к результату работы метода mode() по индексу 0.

Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях):

In [111]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

Из результатов становится ясно, что наибольшее количество продаж недвижимости приходится на май, июль и сентябрь (пятый, седьмой и девятый месяцы соответственно). Месяцами застоя при этом являются месяцы — октябрь, февраль и январь (десятый, второй и первый месяцы соответственно).

## Работа с интервалами

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime:

In [112]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу timedelta.

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

In [113]:
display(delta_days.dt.days)


0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца AgeBuilding:

In [114]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, так как, зная год постройки здания, мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, так как он является более наглядным, а год постройки удалим из таблицы:

In [115]:
melb_df = melb_df.drop('YearBuilt', axis=1)


Создайте в таблице melb_df признак WeekdaySale (день недели). Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье), результат занесите в переменную weekend_count. В качестве ответа введите результат вывода переменной weekend_count.

In [116]:
# Импортируем Pandas
import pandas as pd

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
print(weekend_count)

12822


In [173]:
import pandas as pd

ufo_pd = pd.read_csv('data/ufo.txt', sep=',')
display(ufo_pd)

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00
...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45


In [174]:
# приводим столбец к типу datatime
ufo_pd['Time'] = pd.to_datetime(ufo_pd.Time)

In [ ]:
# выводим на экран полученный столбец
print(ufo_pd['Time'])

In [175]:
# изменяем формат даты 
ufo_pd['Time'] = pd.to_datetime(ufo_pd['Time'], dayfirst=True)
# выводим на экран для проверки, что формат даты изменен
display(ufo_pd['Time'])

0       1930-06-01 22:00:00
1       1930-06-30 20:00:00
2       1931-02-15 14:00:00
3       1931-06-01 13:00:00
4       1933-04-18 19:00:00
                ...        
18236   2000-12-31 23:00:00
18237   2000-12-31 23:00:00
18238   2000-12-31 23:45:00
18239   2000-12-31 23:45:00
18240   2000-12-31 23:59:00
Name: Time, Length: 18241, dtype: datetime64[ns]

In [168]:
# Выводим год с наибольшим числом случаев наблюдения НЛО 
print(ufo_pd['Time'].dt.year.mode()[0])

1999


In [176]:
# Выделяем дату из столбца
print(ufo_pd['Time'].dt.date)

0        1930-06-01
1        1930-06-30
2        1931-02-15
3        1931-06-01
4        1933-04-18
            ...    
18236    2000-12-31
18237    2000-12-31
18238    2000-12-31
18239    2000-12-31
18240    2000-12-31
Name: Time, Length: 18241, dtype: object


In [178]:
ufo_pd['Date'] = ufo_pd['Time'].dt.date


0        1930-06-01
1        1930-06-30
2        1931-02-15
3        1931-06-01
4        1933-04-18
            ...    
18236    2000-12-31
18237    2000-12-31
18238    2000-12-31
18239    2000-12-31
18240    2000-12-31
Name: Date, Length: 18241, dtype: object


In [179]:
# Вычисляем разницу между двумя соседними датами при помощи diff()
# Переводим интервал времени в дни при помощи атрибута timedelta days
print(ufo_pd[ufo_pd['State'] == 'NV']['Date'].diff().dt.days.mean())

68.92932862190813


In [ ]:
# Если необходимо ответ округлить до целого числа:

Библиотека Pandas предоставляет большое количество возможностей для преобразований данных за счёт использования пользовательских функций. 

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода apply(). В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.

Рассмотрим пример. В наших данных есть столбец с адресами объектов недвижимости. Проблема этого столбца в том, что в нём слишком большое количество уникальных значений: почти на каждый объект недвижимости в таблице приходится свой уникальный адрес. 

In [25]:
# Импортируем Pandas
import pandas as pd

# прочитаем наш csv-файл в DataFrame
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')

# Убедимся в этом, вычислив количество уникальных значений в столбце с помощью метода nunique():
print(melb_data['Address'].nunique())

13378


Если мы прогнозируем цену объекта, то такое большое количество возможных категорий может плохо сказаться на модели, которую мы бы хотели в дальнейшем построить на наших данных. Говорят, что такой признак, скорее всего, не имеет статистической значимости, потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку.


Обычно подобные признаки удаляют, однако можно поступить умнее: давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар). Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address:

In [4]:
print(melb_data['Address'].loc[177])
print(melb_data['Address'].loc[1812])
print(melb_data['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [ ]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

# Теперь применим эту функцию к столбцу c адресом. 
# Для этого передадим функцию get_street_type в аргумент метода столбца apply(). 
# В результате получим объект Series, который положим в переменную street_types:
street_types = melb_data['Address'].apply(get_street_type)
display(street_types)

Важно, что функция пишется для одного элемента столбца, а метод apply() применяется к каждому его элементу. Используемая функция обязательно должна иметь возвращаемое значение.

In [10]:
# Посмотрим, сколько уникальных значений у нас получилось:
print(street_types.nunique())

56


In [ ]:
# Частота каждого подтипа улицы с помощью метода value_counts:
display(street_types.value_counts())

Выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' (другие).

Для этого к результату метода value_counts применим метод nlargest(), который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов.

In [27]:
# Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes:
popular_stypes = street_types.value_counts().nlargest(10).index
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

In [39]:
melb_data['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_data['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [29]:
# Посмотрим на результирующее число уникальных подтипов:
print(melb_data['StreetType'].nunique())

11


Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. 

In [30]:
# Удалим Address из нашей таблицы:
melb_data = melb_data.drop('Address', axis=1)

Если присмотреться, то в списке подтипов улиц street_types можно заметить подтипы, которые именуются различным образом, но при этом обозначают одинаковые вещи. Например, подтипы Av и Avenue, Bvd и Boulevard, Pde и Parade. Мы упустили данный момент, хотя в реальных задачах стоит обращать пристальное внимание на результаты преобразований и исправлять неточности в данных.

Такие ошибки в данных (обозначение идентичных категорий различными именами) являются одним из видов «грязных» данных.

Порой отследить такие неточности бывает очень сложно, а при наличии большого количества категорий (например, более ста) — практически невозможно.

## Oбщиe рекомендации по уменьшению числа уникальных значений в признаке, который описывается категориями:

# 1
Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.

# 2
Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).

Далее переходите к шагу 3.

# 3
Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.

# 4
Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.

Когда вы выбрали оптимальное число, переходите к шагу 5.

# 5
Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».

In [36]:
# Импортируем Pandas
import pandas as pd

# В столбце меняем формат даты для возможности выделить день недели
melb_data['Date'] = pd.to_datetime(melb_data['Date'], dayfirst=True)
# Выводим только дни недели 
melb_data['WeekdaySale'] = melb_data['Date'].dt.dayofweek
# Считаем только количество выходных дней
weekend_count = melb_data[(melb_data['WeekdaySale'] == 5) | (melb_data['WeekdaySale'] == 6)].shape[0]
print(weekend_count)

12822


In [38]:
# На вход данной функции поступает элемент столбца.
def get_weekend(weekday):
    # Eсли день является выходным
    if weekday == 5 or weekday == 6:
        # Bозвращает 1
        return 1
    # B противном случае 
    else:
        return 0
    
# Создаём столбец Weekend в таблице melb_df с помощью get_weekend.    
melb_data['Weekend'] = melb_data['WeekdaySale'].apply(get_weekend)
# Применим эту функцию к столбцу 
# и вычислим среднюю цену объекта недвижимости, 
# проданного в выходные дни. Результат округлим до целых.
print(round(melb_data[melb_data['Weekend']==1]['Price'].mean(), 2))


1081198.64


In [ ]:
# Импортируем Pandas
import pandas as pd

# прочитаем наш csv-файл в DataFrame
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_data

In [5]:
# Частота каждого наименованиями риелторских компаний с помощью метода value_counts:
melb_data['SellerG'].value_counts()
# Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_sellers:
popular_sellers = melb_data['SellerG'].value_counts().nlargest(49).index
# Преобразуем столбец SellerG и оставим в столбце только 49 самых популярных
melb_data['SellerG'] = melb_data['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')
# display(melb_data['SellerG'])
# Минимальная цена объектов недвижимости, проданных компанией 'Nelson'
a = melb_data[melb_data['SellerG'] == 'Nelson']['Price'].min() 
# Минимальная цена объектов недвижимости, проданных компаниями, обозначенными как 'other'
b = melb_data[melb_data['SellerG'] == 'other']['Price'].min()
# Найдем, во сколько 'a' раз больше 'b'
print(round(a/b, 1))

1.3


In [ ]:
import pandas as pd

series_2 = pd.Series([
    'Опыт работы 5 лет',
    'Опыт работы 5 месяцев',
    'Опыт работы 1 год 1 месяц',
    'Опыт работы 3 месяца',
    'Опыт работы 7 лет'
])

def get_experience(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
    return int(year)*12 + int(month)

print(series_2.apply(get_experience))

In [ ]:
import pandas as pd
series_1= pd.Series([
    'Опыт работы 8 лет 3 месяца',
    'Опыт работы 3 года 5 месяцев',
    'Опыт работы 1 год 9 месяцев',
    'Опыт работы 3 месяца',
    'Опыт работы 6 лет'
])

def get_experience(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
    return int(year)*12 + int(month)

print(series_1.apply(get_experience))

## Признаки: категориальные и числовые

Числовые признаки (количество) могут быть:

* дискретными (например, количество комнат, пациентов, дней, отток сотрудников);
* непрерывными (например, масса, цена, площадь).

Дискретные признаки чаще всего представлены целыми числами, а непрерывные — целыми числами и числами с плавающей точкой.

Категориальные признаки (классу/категория) могут быть:

* номинальными (например, пол, национальность, район);
* порядковыми (например, уровень образования, уровень комфорта, стадия заболевания).

Такие признаки имеют ограниченный набор значений. Они чаще всего представлены в виде текстового описания и кодируются в Pandas типом данных object.

Однако это не всегда так. Например, созданный нами ранее признак месяца продажи кодируется числом (от 1 до 12), но на самом деле является категориальным, поскольку диапазон его значений ограничен и каждому числу мы можем поставить в соответствие название месяца.

Более детально о классификации признаков здесь https://medium.com/nuances-of-programming/%D1%81%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5-%D1%82%D0%B8%D0%BF%D1%8B-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85-%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D1%83%D0%B5%D0%BC%D1%8B%D0%B5-%D0%B2-%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%BC-%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B8-b8891039b09

## Категории в данных недвижимости 

Определим число уникальных категорий в каждом столбце нашей таблицы melb_df. Для этого создадим вспомогательную таблицу unique_counts:

In [ ]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_data.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_data[col].nunique(),melb_data[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

**1** Создаём пустой список, в который будем добавлять кортежи: имя столбца, количество уникальных значений в нём и тип столбца.

**2** В цикле перебираем имена столбцов, которые получаем с помощью атрибута columns. В переменной col на каждой итерации находятся имена столбцов — обращаемся к ним в цикле и извлекаем число уникальных элементов с помощью метода nunique(), а также тип столбца с помощью атрибута dtypes. Результат заносим в кортеж и добавляем его в список.

**3** Из списка с кортежами (имя столбца, количество уникальных значений в нём, тип столбца) создаём DataFrame, даём названия его столбцам: Column_Name, Num_unique и Type.

**4** Сортируем таблицу по столбцу Num_unique в порядке возрастания количества уникальных элементов с помощью метода sort_values() и выводим результат на экран.

## Tип данных category

Этот тип данных является гибридным: внешне он выглядит как строка, но внутренне представлен массивом целых чисел. 

Самый простой способ преобразования столбцов к типу данных category — это использование уже знакомого нам метода astype(), в параметры которого достаточно передать строку 'category'.

In [25]:
# Bыведим информацию о памяти, занимаемой текущей таблицей:
display(melb_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          13580 non-null  int64  
 1   Suburb         13580 non-null  object 
 2   Address        13580 non-null  object 
 3   Rooms          13580 non-null  int64  
 4   Type           13580 non-null  object 
 5   Price          13580 non-null  float64
 6   Method         13580 non-null  object 
 7   SellerG        13580 non-null  object 
 8   Date           13580 non-null  object 
 9   Distance       13580 non-null  float64
 10  Postcode       13580 non-null  int64  
 11  Bedroom        13580 non-null  int64  
 12  Bathroom       13580 non-null  int64  
 13  Car            13580 non-null  int64  
 14  Landsize       13580 non-null  float64
 15  BuildingArea   13580 non-null  float64
 16  YearBuilt      13580 non-null  int64  
 17  CouncilArea    12211 non-null  object 
 18  Lattit

None

In [26]:
# Сделаем преобразование столбцов к типу данных category:
# список столбцов, которые мы не берём во внимание
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] 
# задаём максимальное число уникальных категорий
max_unique_count = 150 
# цикл по именам столбцов
for col in melb_data.columns: 
    # проверяем условие
    if melb_data[col].nunique() < max_unique_count and col not in cols_to_exclude:  
        # преобразуем тип столбца
        melb_data[col] = melb_data[col].astype('category')
display(melb_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  category
 17  CouncilArea    12211 non-null  

None

**1** Задаём список столбцов, которые мы не берём в рассмотрение (cols_to_exclude), а также условленный нами ранее порог уникальных значений столбца max_unique_count.

**2** В цикле перебираем имена столбцов, и, если число уникальных категорий меньше заданного порога и имён столбцов нет в списке cols_to_exclude, то с помощью метода astype() приводим столбец к типу данных category.

**3** Итоговый объём памяти — 1.9 Мб. В результате такого преобразования объём памяти, занимаемый таблицей, уменьшился почти в 1.5 раза. 

*ПОЛУЧЕНИЕ АТРИБУТОВ CATEGORY*

У типа данных category есть свой специальный аксесcор cat, который позволяет получать информацию о своих значениях и преобразовывать их. 

In [37]:
# C помощью атрибута аксессора cat categories 
# мы можем получить список уникальных категорий в столбце Regionname:

print(melb_data['Regionname'].cat.categories)

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


In [38]:
# Как столбец кодируется в виде чисел в памяти компьютера:
display(melb_data['Regionname'].cat.codes)

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

С помощью метода аксессора rename_categories() можно легко переименовать текущие значения категорий. Для этого в данный метод нужно передать словарь, ключи которого — старые имена категорий, а значения — новые.

Рассмотрим на примере: переименуем категории признака типа постройки Type — заменим их на полные названия (напомним, u — unit, h — house, t — townhouse).

In [39]:
melb_data['Type'] = melb_data['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_data['Type'])

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

* А может что-то пойти не так?

In [40]:
# Допустим, у нас новая партия домов и теперь мы продаём и квартиры (flat). 
# Создадим объект Series new_houses_types, где будем хранить типы зданий новой партии домов. 
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
# Преобразуем тип new_houses_types в такой же тип, как и у столбца Type в таблице melb_data
new_houses_types = new_houses_types.astype(melb_data['Type'].dtype)
# Bыведем результат на экран
display(new_houses_types)


0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

Mы получили пустые значения — NaN, потому что тип данных category хранит только категории, которые были объявлены при его инициализации. Поэтому для решения проблемы:

In [ ]:
# Добавим категорию flat в столбец Type с помощью метода акссесора cat add_categories(), 
# в который достаточно просто передать имя новой категории:
melb_data['Type'] = melb_data['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_data['Type'].dtype)
display(new_houses_types)

Добавление новой категории в столбец Type не отразится на самом столбце — текущие категории не изменятся, однако такое преобразование позволит добавлять в таблицу новые данные о домах с новой категорией — flat.

In [46]:
# Узнаём, сколько памяти теперь занимает таблица melb_df.
melb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  category
 17  CouncilArea    12211 non-null  

In [ ]:
# Частота каждого наименования популярных пригородов с помощью метода value_counts:
melb_data['Suburb'].value_counts()
# Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_suburb:
popular_suburb = melb_data['Suburb'].value_counts().nlargest(119).index
# Преобразуем столбец Suburb и оставим в столбце только 119 самых популярных
melb_data['Suburb'] = melb_data['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')
new_melb_data = melb_data.astype(melb_data['Suburb'].dtype)
new_melb_data.info()


In [74]:
# Импортируем Pandas
import pandas as pd

# прочитаем наш csv-файл в DataFrame
citibike_tripdata = pd.read_csv('data/citibike-tripdata.csv', sep=',')
citibike_tripdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

Датасет представляет собой таблицу с информацией о 300 тысячах поездок за первые пять дней сентября 2018 года и включает в себя следующую информацию:

* starttime — время начала поездки (дата, время);
* stoptime — время окончания поездки (дата, время);
* start station id — идентификатор стартовой стоянки;
* start station name — название стартовой стоянки;
* start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;
* end station id — идентификатор конечной стоянки;
* end station name — название конечной стоянки;
* end station latitude, end station longitude — географическая широта и долгота конечной стоянки;
* bikeid — идентификатор велосипеда;
* usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);
* birth year — год рождения клиента;
* gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина).

In [51]:
# Идентификатор самой популярной стартовой стоянки. 
# Запишите идентификатор в виде целого числа.
citibike_tripdata['start station id'].mode()

0    281.0
Name: start station id, dtype: float64

In [52]:
# Велосипед с каким идентификатором является самым популярным
citibike_tripdata['bikeid'].mode()

0    33887
Name: bikeid, dtype: int64

In [75]:
# преобладающий тип клиентов
mode_usertype = citibike_tripdata['usertype'].mode()[0]
# Произвoдим фильтрацию и наxoдим поездки преобладающего типа клиентов.
count_mode_user = citibike_tripdata[citibike_tripdata['usertype'] == mode_usertype].shape[0]
# доля клиентов преобладающего типа среди общего количества клиентов
# делим число строк в отфильтрованной таблице на общее число строк.
print(round(count_mode_user / citibike_tripdata.shape[0], 2))

0.77


In [76]:
# Кто больше занимается велоспортом — мужчины или женщины?
display(citibike_tripdata['gender'].value_counts())

1    183582
2     74506
0     41912
Name: gender, dtype: int64

In [77]:
# Кто больше занимается велоспортом — мужчины или женщины?
# В ответ запишите число поездок для той группы, у которой их больше.
male_count = citibike_tripdata[citibike_tripdata['gender'] == 1].shape[0]
female_count = citibike_tripdata[citibike_tripdata['gender'] == 0].shape[0]
print(max([male_count, female_count]))

183582


In [78]:
# Число уникальных стартовых и конечных стоянок, которыми воспользовались клиенты
print(citibike_tripdata['start station id'].nunique())
print(citibike_tripdata['end station id'].nunique())


759
765


In [79]:
# минимальный возраст клиента
age = 2018 - citibike_tripdata['birth year']
print(age.min())

16


In [ ]:
# Самой непопулярной стартовой стоянкой из тех, 
# которыми воспользовались клиенты, является стоянка с названием 
# "Eastern Pkwy & Washington Ave"
display(citibike_tripdata['start station name'].value_counts())

In [ ]:
# Наибольшее количество поездок завершается на стоянке под названием
display(citibike_tripdata['end station name'].value_counts())

In [ ]:
# Удалим признаки идентификаторов стоянок
citibike_tripdata.drop(['start station id','end station id'],axis=1,inplace=True)
citibike_tripdata.info()

In [72]:
# Заменим признак birth year на более понятный признак возраста клиента age
# Годом отсчёта возраста выберите 2018 год
citibike_tripdata['age'] = 2018 - citibike_tripdata['birth year']
# Столбец birth year удалим из таблицы
citibike_tripdata.drop(['birth year'], axis=1, inplace=True)
# Вычислим число поездок совершенных клиентами старше 60 лет
print(citibike_tripdata[citibike_tripdata['age'] > 60].shape[0])

11837


In [80]:
# Приводим столбцы starttime и stoptime в формат datetime
citibike_tripdata['starttime'] = pd.to_datetime(citibike_tripdata['starttime'])
citibike_tripdata['stoptime'] = pd.to_datetime(citibike_tripdata['stoptime'])
# Вычислим длительность поездки рассчитывается как разница между stoptime и starttime. 
# Результат заносим в столбец trip duration. 
citibike_tripdata['trip duration'] = (citibike_tripdata['stoptime'] - citibike_tripdata['starttime'])
citibike_tripdata.loc[3, 'trip duration']

Timedelta('0 days 00:07:16.837000')

In [84]:
# Создайте «признак-мигалку» weekend, который равен 1, 
# если поездка начиналась в выходной день (суббота или воскресенье), 
# и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.
weekday = citibike_tripdata['starttime'].dt.dayofweek
citibike_tripdata['weekend'] = weekday.apply(lambda x: 1 if x ==5 or x == 6 else 0)
citibike_tripdata['weekend'].sum()

115135

In [87]:
def get_time_of_day(time):
    # поездка совершается ночью (night), если её час приходится 
    # на интервал от 0 (включительно) до 6 (включительно) часов
    if 0 <= time <= 6:
        return 'night'
    # поездка совершается утром (morning), если её час приходится
    # на интервал от 6 (не включительно) до 12 (включительно) часов
    elif 6 < time <= 12:
        return 'morning'
    # поездка совершается днём (day), если её час приходится 
    # на интервал от 12 (не включительно) до 18 (включительно) часов
    elif 12 < time <= 18:
        return 'day'
    # поездка совершается вечером (evening), если её час приходится 
    # на интервал от 18 (не включительно) до 23 часов (включительно)
    elif 18 < time <= 23:
        return 'evening'
    else:
        return 'else'
# Создаем признак времени суток поездки time_of_day. 
# Время суток будем определять из часа начала поездки.
citibike_tripdata['time_of_day'] = citibike_tripdata['starttime'].dt.hour.apply(get_time_of_day)
# количество поездок, совершённых днём
a = citibike_tripdata[citibike_tripdata['time_of_day'] == 'day'].shape[0]
# количество поездок, совершённых ночью
b = citibike_tripdata[citibike_tripdata['time_of_day'] == 'night'].shape[0]
# Во сколько раз количество поездок больше
print(round(a / b))

9


Статья «Feature engineering: шесть шагов для создания успешной модели машинного обучения» https://hightech.fm/2018/08/09/featureengineering